### Importação de blibiotecas

In [ ]:
import pandas as pd; pd.set_option('display.max_rows', 20); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Importação das tabelas

In [ ]:
df_1 = pd.read_csv('BaseDados_ProjetoINTELI_RG_31JUL24.csv')
df_2 = pd.read_csv('BaseDados_ProjetoINTELI_RG_19JUL24.csv')
df_3 = pd.read_csv('BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv')

In [ ]:
pd.set_option('display.max_rows', 10)


## Tratamento Tabela 1

### Primeira análise da tabela

In [ ]:
list(df_1.columns)

### Drop das colunas divergentes das outras tabelas
a coluna 'Mês/ano', seria uma coluna repetida e sem uso na tabela, isso porque temos as colunas 'Mês' e 'ano', que além de estarem separadas estão mais facilitadas para a conversão de formato Time (Data). A Coluna 'CONCATENAR' são muitos dados nulos assim como a coluna '% Ating. Meta'.

In [ ]:
df_1 = df_1.drop(columns=['Mês/ano', 'CONCATENAR', '% Ating. Meta'])
df_1 = df_1.rename(columns={'Ano':'ano'})

In [ ]:
df_1.head()

### Checagem de dados nulos
Temos mesmo com o drop das colunas divergentes vários dados nulos na tabela 1, por isso devemos identificá-los e acharmos alguma forma de preenche-los.

In [ ]:
df_1.isna().sum()

Aqui vemos que os dois tipos de setor que estão nulos na tabela 1, são Comércios de acordo com os tipos de classificação possíveis nessa coluna ('Setor').

In [ ]:
df_1[df_1['Setor'].isna()]

In [ ]:
df_1['Setor'].value_counts()

In [ ]:
df_1.at[36839, 'Setor'] = 'COMERC.P/ AUTOMOTIVO'
df_1.at[45332, 'Setor'] = 'LOJA VAREJISTA/ATAC'

Agora iremos ordenar a tabela pela data para podermos completar os dados nulos de PIB

In [ ]:
df_1 = df_1.sort_values(by=['ano', 'Mês'])
df_1

In [ ]:
df_1 = df_1.fillna(method='ffill')

In [ ]:
df_1.isna().sum()

### Arrumar erros de digitação/formatação
A coluna desconto não deve possuir uma porcentagem negativa e sim positiva, então removemos o '-' de todos as porcentagens de desconto

In [ ]:
# df.loc[(df['Cliente'] == 'DMA DISTRIBUIDORA SA') & (df['ano'] == 2023) & (df['Mês'] == 1)]
df_1['Desc %'] = df_1['Desc %'].str.replace('-','')


### Apagar linhas duplicadas

In [ ]:
df_1.drop_duplicates(inplace=True)
df_1.duplicated().sum()

In [ ]:
# df_4.drop(columns=['EMPRESA', 'NOME EMPRESA', 'UN', 'VEÍCULO', 'AUTORIZAÇÃO', 'DT AUTORIZAÇÃO', 'CLIENTE', 'NOME FANTASIA CLIENTE', 'GRUPO CLIENTE', 'SETOR', 'SUBSETOR', 'NOME SUBSETOR', 'AGÊNCIA', 'NOME AGÊNCIA', 'EQUIPE CV', 'CANAL DE VENDA', 'NOME CANAL DE VENDA', 'TÍTULO', 'É PATROCICIO', 'CONTRATO', 'DT INICIAL', 'DT FINAL', 'TIPO MÍDIA', 'DT EVENTO', 'TP PI', '' ])

### Transformação dos tipos de dados númericos de string para integers/números

In [ ]:
uen = df_1.sort_values(by=['Cliente', 'Veiculo'])
# uen[uen['Cliente'] == '']

In [ ]:
#Limpando a Coluna de Valor liquído e Convertendo colunas de string para número
df_1['Vl Liquido Final'] = df_1['Vl Liquido Final'].str.replace('.', '')
df_1['Vl Liquido Final'] = df_1['Vl Liquido Final'].str.replace(',', '.')
df_1['Vl Bruto'] = df_1['Vl Bruto'].str.replace('.', '')
df_1['Vl Bruto'] = df_1['Vl Bruto'].str.replace(',', '.')
df_1['Desconto R$'] = df_1['Desconto R$'].str.replace('.', '')
df_1['Desconto R$'] = df_1['Desconto R$'].str.replace(',', '.')
df_1['VL Tabela'] = df_1['VL Tabela'].str.replace('.', '')
df_1['VL Tabela'] = df_1['VL Tabela'].str.replace(',', '.')
df_1['IPCA ES'] = df_1['IPCA ES'].str.replace(',', '.')
df_1['IPCA BR'] = df_1['IPCA BR'].str.replace(',', '.')
df_1['Taxa Ac. TRI % PIB'] = df_1['Taxa Ac. TRI % PIB'].str.replace(',', '.')
df_1['Desc %'] = df_1['Desc %'].str.replace('%', '')
#to_numeric
df_1['Vl Liquido Final'] = pd.to_numeric(df_1['Vl Liquido Final'])
df_1['Vl Bruto'] = pd.to_numeric(df_1['Vl Bruto'])
df_1['Desconto R$'] = pd.to_numeric(df_1['Desconto R$'])
df_1['VL Tabela'] = pd.to_numeric(df_1['VL Tabela'])
df_1['IPCA ES'] = pd.to_numeric(df_1['IPCA ES'])
df_1['IPCA BR'] = pd.to_numeric(df_1['IPCA BR'])
df_1['Taxa Ac. TRI % PIB'] = pd.to_numeric(df_1['Taxa Ac. TRI % PIB'])
df_1['Desc %'] = pd.to_numeric(df_1['Desc %'])
df_1['Desc %'] = df_1['Desc %']/100
df_1['ano'] = pd.to_numeric(df_1['ano'])
df_1['Mês'] = pd.to_numeric(df_1['Mês'])

### Tranformação das colunas de Dia, Mês e Ano para formato de Time (Data)

In [ ]:
#Criando coluna 'Dia' para poder converter conjuntamente com as colunas 'Mês' e 'ano' para um formato Dataframe
df_1['Dia'] = 1
df_1_to_time = df_1[['ano', 'Mês', 'Dia']].rename(columns={'ano': 'year', 'Mês': 'month', 'Dia': 'day'})
df_1['Data'] = pd.to_datetime(df_1_to_time)
#Apagando as colunas repetidas que já estão presente no Dataframe
df_1.drop(columns=['ano', 'Mês', 'Dia'], inplace=True)
df_1

## Tratamento Tabela 2

### Checagem dos dados nulos
e preenchimento dos mesmo conforme a classificação possível do setor

In [ ]:
df_2.isna().sum()

In [ ]:
df_2[df_2['Setor'].isna()]

In [ ]:
df_2.at[34853, 'Setor'] = 'COMERC.P/ AUTOMOTIVO'
df_2.at[56106, 'Setor'] = 'LOJA VAREJISTA/ATAC '
df_2.isna().sum()

In [ ]:
df_2.head()

### Transformação dos tipos de dados númericos de string para integers/números

In [ ]:
#Limpando a Coluna de Valor liquído e Convertendo colunas de string para número
df_2['Vl Liquido Final'] = df_2['Vl Liquido Final'].str.replace('.', '')
df_2['Vl Liquido Final'] = df_2['Vl Liquido Final'].str.replace(',', '.')
df_2['Vl Bruto'] = df_2['Vl Bruto'].str.replace('.', '')
df_2['Vl Bruto'] = df_2['Vl Bruto'].str.replace(',', '.')
df_2['Desconto R$'] = df_2['Desconto R$'].str.replace('.', '')
df_2['Desconto R$'] = df_2['Desconto R$'].str.replace(',', '.')
df_2['VL Tabela'] = df_2['VL Tabela'].str.replace('.', '')
df_2['VL Tabela'] = df_2['VL Tabela'].str.replace(',', '.')
df_2['Desc %'] = df_2['Desc %'].str.replace('%', '')
#to_numeric
df_2['Vl Liquido Final'] = pd.to_numeric(df_2['Vl Liquido Final'])
df_2['Vl Bruto'] = pd.to_numeric(df_2['Vl Bruto'])
df_2['Desconto R$'] = pd.to_numeric(df_2['Desconto R$'])
df_2['VL Tabela'] = pd.to_numeric(df_2['VL Tabela'])
df_2['Desc %'] = pd.to_numeric(df_2['Desc %'])
df_2['Desc %'] = df_2['Desc %']/100
df_2['ano'] = pd.to_numeric(df_2['ano'])
df_2['Mês'] = pd.to_numeric(df_2['Mês'])

### Tranformação das colunas de Dia, Mês e Ano para formato de Time (Data)

In [ ]:
#Criando coluna 'Dia' para poder converter conjuntamente com as colunas 'Mês' e 'ano' para um formato Dataframe
df_2['Dia'] = 1
df_2_to_time = df_2[['ano', 'Mês', 'Dia']].rename(columns={'ano': 'year', 'Mês': 'month', 'Dia': 'day'})
df_2['Data'] = pd.to_datetime(df_2_to_time)
#Apagando as colunas repetidas que já estão presente no Dataframe
df_2.drop(columns=['ano', 'Mês', 'Dia'], inplace=True)
df_2

### Checagem de linhas duplicadas

In [ ]:
df_2.duplicated().sum()

##  Tratamento Tabela 3

### Tratamento para tirar os Nulos

In [ ]:
# Visualização da quantidade de valores nulos em df_3
print('Valores Nulos:\n', df_3.isna().sum())


In [ ]:
# Visualização das linhas com valores nulos
df_3[df_3['Setor'].isna()]

In [ ]:
# Preenchimento dos valores nulos
df_3.at[36839, 'Setor'] = 'COMERC.P/ AUTOMOTIVO'
df_3.at[45332, 'Setor'] = 'LOJA VAREJISTA/ATAC '


### Tratamento de duplicatas

In [ ]:
df_3.duplicated().sum()

In [ ]:
df_3.drop_duplicates(inplace=True)
df_3.duplicated().sum()

### Tratando a coluna de desconto, tirando o "-" dos valores

In [ ]:
df_3['Desc %'] = df_3['Desc %'].astype(str).str.replace('-','')

### Definindo as colunas que são numéricas e tornando elas de fato números

In [ ]:
    for column in ['Vl Liquido Final', 'Vl Bruto', 'Desconto R$', 'VL Tabela', 'Desc %']:
        df_3[column] = df_3[column].str.replace('.', '', regex=False) 
        df_3[column] = df_3[column].str.replace(',', '.', regex=False)
        df_3[column] = df_3[column].str.replace('%', '', regex=False)
        df_3[column] = pd.to_numeric(df_3[column])                     

df_3.head(3)

### Tornando todas as informações referentes a data uma coluna só

In [ ]:
df_3['Dia'] = 1
df_3_to_time = df_3[['ano', 'Mês', 'Dia']].rename(columns={'ano': 'year', 'Mês': 'month', 'Dia': 'day'})
df_3['Data'] = pd.to_datetime(df_3_to_time)
#Apagando as colunas repetidas que já estão presente no Dataframe
df_3.drop(columns=['ano', 'Mês', 'Dia'], inplace=True)


In [ ]:
df_3['UEN'] = df_3['UEN'].str.replace('DIGITAL', 'Digital')

In [ ]:
df_3['UEN'].value_counts()

In [ ]:
df_3.head()

## Tratamento das tabelas 4 e 5

#### Processamento das tabelas


In [ ]:
df_4 = pd.read_csv('DadosInternos_RedeGazeta_Inteli v2.csv')
df_5 = pd.read_csv('DadosInternos_RedeGazeta_Inteli.csv')

#### Vizualização das colunas das tabelas 
Para entender a similaridade dos parâmetros das tabelas é preciso a vizualização das colunas


In [ ]:
# Visualização das colunas das tabelas 4 e 5
print('df_4 columns', df_4.columns)
print('df_5 columns', df_5.columns)

In [ ]:
# Verificação de valores nulos
print('Is na', df_4.isna().sum())
print('Is na', df_5.isna().sum())

#### Eliminação das colunas irrelevantes para a análise de dados
Após o entendimento das colunas e para evitar deficiência da análise, o próximo passo é a eliminação das colunas com diversos valores nulos

In [ ]:
# Eliminação das colunas irrelevantes para a análise de dados
df_4.drop(['TIPO DE RECEITA', 'TIPO DE RECEITA 2','Unnamed: 38'],inplace=True, axis=1)
df_5.drop(['TIPO DE RECEITA', 'TIPO DE RECEITA 2','t'],inplace=True, axis=1)

In [ ]:
# Após a retirada das colunas, visualização da quantidade de linhas nas tabelas
print('df_4 len', len(df_4))
print('df_5 len', len(df_5))

In [ ]:
# Visualização das colunas após o tratamento
print('df_4.columns:', df_4.columns)
print('df_5.columns:',df_5.columns)

#### Quantificar dados nulos em cada coluna da tabela 
Inspecionar os dados e coletar quais estão nulos permite o entender quais colunas são pertinentes para a análise

In [ ]:
# Quantidade de valores nulos na tabela 4
print('Is na', df_4.isna().sum())

In [ ]:
# Quantidade de valores nulos na tabela 5
print('Is na', df_5.isna().sum())

#### Quantificar linhas duplicadas
É comum linhas duplicadas em grandes tabelas de dados para evitar overfitting

In [ ]:
# Verificando quantas linhas duplicadas tem na tabela df_4
df_4.duplicated().sum()

In [ ]:
# Verificando quantas linhas duplicadas tem na tabela df_5
df_5.duplicated().sum()

In [ ]:
# Eliminando linhas duplicadas nas tabelas
df_4.drop_duplicates(inplace=True)
df_5.drop_duplicates(inplace=True)

In [ ]:
# Verificação de linhas duplicadas após tratamento
print(df_4.duplicated().sum())
print(df_5.duplicated().sum())

In [ ]:
# Vizualização da quantidade de colunas nas tabelas 4 e 5
print(len(df_4.columns))
print(len(df_5.columns))

In [ ]:
# Vizualização da quantidade de linhas das tabelas
print('df_4 len', len(df_4))
print('df_5 len', len(df_5))

In [ ]:
df_4.columns

In [ ]:
df_5.columns

In [ ]:
# Vizualização das 5 primeiras linhas da tabela 4
df_4.head()

In [ ]:
# Vizualização das 5 primeiras linhas da tabela 5
df_5.head()

## Concatenação das tabelas 4 e 5
Após a validação do tratamento das duas tabelas, é pertinente uni-las

In [ ]:
# União das duas tabelas em uma
df_clean2 = pd.concat([df_4, df_5])

Ao concatener, foi somado 41560 linhas duplicadas, o mesmo comprimento das tabelas antes da junção

In [ ]:
# Verificação de valores duplicados na tabela concatenada
df_clean2.duplicated().sum()

In [ ]:
# Eliminação de todas as linhas duplicadas
df_clean2.drop_duplicates(inplace=True)

In [ ]:
# Validação da eliminação dos valores duplicados
df_clean2.duplicated().sum()

In [ ]:
df_clean2.columns

In [ ]:
df_clean2['% DESCONTO/ACRÉSCIMO'].describe

#### Conclusão
Os dados da tabela 4 e 5 estão duplicados, uma vez que o a quantidade de valores duplicados é o mesmo que a quantidade de linhas das tabelas

In [ ]:
# Visualização das primeiras 5 colunas da tabela concatenada
df_clean2.head()

## Tratamento de dados após concatenar as tabelas
A seguir, mudar o tipo de dado de object para int ou float. Para isso, coletar as colunas específicas

In [ ]:
# Coletar apenas colunas que tem valores como números porém tipo de dado como 'object'
df_clean2[['EMPRESA',  'UN', 'VEÍCULO', 'AUTORIZAÇÃO', 'CLIENTE',
       'SETOR', 'SUBSETOR', 'AGÊNCIA', 'CANAL DE VENDA', 'CONTRATO', 'VL TABELA',
       'VL DESCONTO/ACRÉSCIMO', '% DESCONTO/ACRÉSCIMO', 'VL BRUTO',
       'VL COMISSÃO', 'VL LÍQUIDO']].info()

In [ ]:
# Colunas para mudar o tipo do dado
erase_comma = df_clean2[['VL TABELA', 'VL DESCONTO/ACRÉSCIMO', '% DESCONTO/ACRÉSCIMO', 'VL BRUTO', 'VL COMISSÃO', 'VL LÍQUIDO']]

#### Elimição das vírgulas para converter o tipo do dados
É possível a a mudança do tipo de dado apenas quando os valores das colunas não tiverem vírgulas

In [ ]:
# Para cada coluna selecionada, trocar vírgula por ponto
for column in erase_comma.columns:
    df_clean2[column] = df_clean2[column].str.replace(',', '.')
    df_clean2[column] = df_clean2[column].str.replace('-', '')
    df_clean2[column] = pd.to_numeric(df_clean2[column], errors='coerce')

In [ ]:
df_clean2['ANO'] = df_clean2['MÊS/ANO'].astype(str).str[:4].astype(int)
df_clean2['MÊS'] = df_clean2['MÊS/ANO'].astype(str).str[4:].astype(int)
df_clean2['DIA'] = 1
df_to_datatime_4 = df_clean2[['ANO', 'MÊS', 'DIA']].rename(columns={'ANO': 'year', 'MÊS': 'month', 'DIA': 'day'})
df_clean2['MÊS/ANO'] = pd.to_datetime(df_to_datatime_4)

#### Verificação da convesão de dados


In [ ]:
# Validação da conversão de dados
df_clean2.info()

## Concact Tabela 3 e 1

In [ ]:
df_IPCA = df_1[['IPCA BR', 'IPCA ES', 'Taxa Ac. TRI % PIB', 'Data']]
print('Antes do drop_duplicates', df_IPCA.duplicated().sum())
df_IPCA.drop_duplicates(inplace=True)
print('Depois do drop_duplicates',df_IPCA.duplicated().sum())

In [ ]:
df_IPCA.columns

In [ ]:
# Alinhamento dos IPCA's com suas respectivas linhas
df = pd.merge(df_3, df_IPCA, on='Data', how='left')

# Organização do Data Frame por Data
df.sort_values(by=['Data'])

# Eliminação de Colunas irrelevantes para a previsão do modelo
df.drop(columns=['Segmento', 'Origem'], inplace=True)

# Visualização dos valores nulos
print('Valores Nulos:', df.isna().sum())

# Eliminação de linhas com valores nulos
df = df.dropna(subset=['Setor'])
df = df.dropna(subset=['Desc %'])

# Verificação de valores duplicados
df[df.duplicated()]
df.drop_duplicates(inplace=True)

## Concatenação

In [ ]:
df_clean2.drop(columns=['EMPRESA', 'NOME EMPRESA', 'UN', 'VEÍCULO', 'AUTORIZAÇÃO', 'DT AUTORIZAÇÃO', 'CLIENTE', 'NOME FANTASIA CLIENTE', 'GRUPO CLIENTE', 'SETOR', 'SUBSETOR', 'NOME SUBSETOR', 'AGÊNCIA', 'NOME AGÊNCIA', 'EQUIPE CV', 'CANAL DE VENDA', 'NOME CANAL DE VENDA', 'TÍTULO', 'É PATROCÍCIO', 'CONTRATO', 'DT INICIAL', 'DT FINAL', 'TIPO MÍDIA', 'DT EVENTO', 'TIPO PI', 'VL COMISSÃO', 'ANO', 'MÊS', 'DIA'], inplace=True)
df_clean2 = df_clean2.rename(columns={'NOME UN': 'UEN', 'NOME VEÍCULO': 'Veiculo', 'MÊS/ANO': 'Data', 
                          'RAZÃO SOCIAL CLIENTE': 'Cliente','NOME SETOR': 'Setor', 'VL TABELA':'VL Tabela', 
                          'VL DESCONTO/ACRÉSCIMO':'Desconto R$', '% DESCONTO/ACRÉSCIMO':'Desc %', 
                          'VL BRUTO':'Vl Bruto', 'VL LÍQUIDO':'Vl Liquido Final'}, copy=False)
print('Columns df_clean2:', df_clean2.columns)


In [ ]:
# Lista dos valores das Unidades de Negócios
print(df_clean2['UEN'].value_counts())

# Padronizando a escrita das Unidade de Negócios
df_clean2['UEN'].replace('RADIO', 'Rádio', inplace=True)
df_clean2['UEN'].replace('TELEVISAO', 'Televisão', inplace=True)
df_clean2['UEN'].replace('JORNAL', 'Digital', inplace=True)
df_clean2['UEN'].replace('DIGITAL', 'Digital', inplace=True)

# Verificação da Padronização
df_clean2['UEN'].value_counts()

In [ ]:
df_6 = pd.read_csv('DadosEconomicos_ES_Inteli.xlsx-VarejoTotal.csv')
df_6 = df_6[df_6['PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)'] != '...']
colunas_numericas = df_6.select_dtypes(include=['float64', 'int64']).columns
outliers = {}
for coluna in colunas_numericas:
    Q1 = df_6[coluna].quantile(0.25)
    Q3 = df_6[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    condicao = (df_6[coluna] < limite_inferior) | (df_6[coluna] > limite_superior)
    outliers[coluna] = df_6[condicao]
if any(not df.empty for df in outliers.values()):
    for coluna, df_outlier in outliers.items():
        if not df_outlier.empty:
            styled = df_outlier.style.set_caption(f"Outliers na coluna {coluna}").set_table_styles([{
                'selector': 'caption',
                'props': [('color', 'red'), ('font-size', '16px')]
            }]).highlight_null(color='yellow').background_gradient(cmap='viridis')
            display(styled)
else:
    print("Não foram encontrados outliers em nenhuma coluna.")

In [ ]:
meses_map = {
    'jan.': 'Jan',
    'fev.': 'Feb',
    'mar.': 'Mar',
    'abr.': 'Apr',
    'mai.': 'May',
    'jun.': 'Jun',
    'jul.': 'Jul',
    'ago.': 'Aug',
    'set.': 'Sep',
    'out.': 'Oct',
    'nov.': 'Nov',
    'dez.': 'Dec'
}
# Converter a coluna de data
df_6['Data'] = pd.to_datetime(df_6['Mês'].replace(meses_map, regex=True).str.replace(r'(\w+)-(\d+)', 
                                                       lambda m: f"01-{m.group(1)}-20{m.group(2)}",
                                                       regex=True),
                            format='%d-%b-%Y', errors='coerce')
# Formatar a data no novo formato padrão (YYYY-MM-DD)
df_6['Data'] = df_6['Data'].dt.strftime('%Y-%m-%d')
df_6.head()

In [ ]:
df_pmc_br = df_6[df_6['Brasil e Unidade da Federação'] == 'Brasil']
df_pmc_es = df_6[df_6['Brasil e Unidade da Federação'] == 'Espírito Santo']
df_pmc_es = df_pmc_es.drop(columns=['Brasil e Unidade da Federação','Tipos de índice', 'Mês'])
df_pmc_br = df_pmc_br.drop(columns=['Brasil e Unidade da Federação','Tipos de índice', 'Data', 'Mês'])


In [ ]:
df_pmc_es.rename(columns={'PMC - Número-índice (2022=100) (Número-índice)': 'PMC - Número-índice (2022=100) (Número-índice)/ ES'}, inplace=True)
df_pmc_es.rename(columns={'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)': 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES'}, inplace=True)
df_pmc_es.rename(columns={'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)': 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES'}, inplace=True)
df_pmc_br.rename(columns={'PMC - Número-índice (2022=100) (Número-índice)': 'PMC - Número-índice (2022=100) (Número-índice)/ BR'}, inplace=True)
df_pmc_br.rename(columns={'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)': 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR'}, inplace=True)
df_pmc_br.rename(columns={'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)': 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR'}, inplace=True)
df_pmc_es = df_pmc_es.reset_index() 
df_pmc_br = df_pmc_br.reset_index()
df_combined = pd.concat([df_pmc_es, df_pmc_br], axis=1)
df_combined = df_combined.drop(columns=['index'])

df_combined = df_combined[['PMC - Número-índice (2022=100) (Número-índice)/ ES',
                           'PMC - Número-índice (2022=100) (Número-índice)/ BR',
                           'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES',
                           'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR',
                           'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES',
                           'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR',
                           'Data']]

df_6 = df_combined


In [ ]:
df_6['Data'] = pd.to_datetime(df_6['Data'])

In [ ]:
df = pd.merge(df, df_6, on='Data', how='left')

In [ ]:
erase_comma_df = df[['PMC - Número-índice (2022=100) (Número-índice)/ ES', 'PMC - Número-índice (2022=100) (Número-índice)/ BR', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR']]

In [ ]:
# Para cada coluna selecionada, trocar vírgula por ponto
for column in erase_comma_df.columns:
    df[column] = df[column].str.replace(',', '.')
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [ ]:
df.head()

## Validação da Tabela 

In [ ]:
df_receita = pd.read_csv('BASE INTELI_META_OCUP.csv')

In [ ]:
df_receita['% Ating. Meta'] = df_receita['% Ating. Meta'].str.replace('%', '')
df_receita['% Ating. Meta'] = pd.to_numeric(df_receita['% Ating. Meta'])
df_receita['% Ating. Meta'] = df_receita['% Ating. Meta']/100

In [ ]:
df_receita['Meta'] = df_receita['Meta'].str.replace('.', '')
df_receita['Meta'] = pd.to_numeric(df_receita['Meta'])
df_receita['Receita'] = df_receita['Meta'] * df_receita['% Ating. Meta']

In [ ]:
for index, mes in enumerate(df_receita['Mês'].unique()):
    df_receita.loc[df_receita['Mês'] == mes, 'Mês'] = index + 1
df_receita.insert(2, 'Dia', 1)
df_receita['Ano'] = pd.to_numeric(df_receita['Ano'], errors='coerce')
df_receita.rename(columns={'Ano': 'year', 'Mês': 'month', 'Dia': 'day'}, inplace=True)
df_receita.insert(0, 'Data', pd.to_datetime(df_receita[['year', 'month', 'day']]))

In [ ]:
from math import sqrt
df.sort_values(['Data'])
df_receita.sort_values(['Data'])
resultados = []
for uen in df['UEN'].unique():
    for data in df['Data'].astype(str).str[:7].unique():
        # Soma das receitas da tabela tratada
        receita_por_veiculo = df.loc[(df['UEN'] == uen) & (df['Data'].astype(str).str[:7] == data), 'Vl Liquido Final'].sum()
        # Receita real por veículo/mês/ano
        receita_por_veiculo_real = df_receita.loc[(df_receita['Uen'] == uen) & (df_receita['Data'].astype(str).str[:7] == data), 'Receita'].values
        # Condição para validar a equivalência dos valores
        resultados.append({
            'Data': data,
            'UEN': uen,
            'Receita Prevista': receita_por_veiculo,
            'Receita Real': np.round(receita_por_veiculo_real, decimals=0),
            'Diferença': np.round(receita_por_veiculo_real - receita_por_veiculo, decimals=0)
        })
df_validacao = pd.DataFrame(resultados)


In [ ]:
df_validacao.sort_values(by=['Data']).reset_index().drop(columns=['index'])
df_validacao.sort_values(by=['Data'])

In [ ]:
df.head()

## Plotagem dos gráficos 

### Definição de coluna Categórica e Numérica

### Introdução
O gráfico de pizza apresentado a seguir ilustra a distribuição das colunas de um DataFrame em duas categorias principais: "Numérico" e "Categórico". O objetivo é visualizar a proporção de variáveis de diferentes tipos de dados presentes no conjunto de dados, permitindo uma compreensão clara da composição dos dados para futuras análises e modelagens. Colunas categóricas representam variáveis qualitativas, enquanto colunas numéricas representam variáveis quantitativas.


### Colunas Categóricas

As colunas categóricas são aquelas que contêm dados de tipo `object`, representando variáveis qualitativas que indicam diferentes categorias ou rótulos. As colunas categóricas identificadas no DataFrame são:

- **UEN**: Unidade de Negócio.
- **Veiculo**: Tipo ou nome do veículo utilizado.
- **Cliente**: Nome do cliente.
- **Setor**: Setor econômico ou categoria a que pertence o cliente.
- **Data**: Data, tratada aqui como uma categoria ao invés de um valor temporal.

### Colunas Numéricas

As colunas numéricas são aquelas que contêm dados de tipos numéricos (`int`, `float`, etc.), representando variáveis quantitativas que podem ser usadas para cálculos matemáticos ou análises estatísticas. As colunas numéricas identificadas no DataFrame são:

- **VL Tabela**: Valor na Tabela.
- **Desconto R$**: Valor de desconto em reais.
- **Desc %**: Desconto em porcentagem.
- **Vl Bruto**: Valor bruto antes de descontos.
- **Vl Liquido Final**: Valor líquido após todos os descontos.
- **IPCA BR**: Índice de Preços ao Consumidor Amplo (Brasil).
- **IPCA ES**: Índice de Preços ao Consumidor Amplo (Espírito Santo).
- **Taxa Ac. TRI % PIB**: Taxa acumulada trimestral do PIB em porcentagem.
- **PMC - Número-índice (2022=100) (Número-índice)/ ES**: Índice de produção e consumo com base em 2022 para o Espírito Santo.
- **PMC - Número-índice (2022=100) (Número-índice)/ BR**: Índice de produção e consumo com base em 2022 para o Brasil.
- **PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES**: Índice de produção e consumo ajustado sazonalmente para o Espírito Santo.
- **PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR**: Índice de produção e consumo ajustado sazonalmente para o Brasil.
- **PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES**: Variação mensal ajustada sazonalmente para o Espírito Santo.
- **PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR**: Variação mensal ajustada sazonalmente para o Brasil.
- **Ano**: Ano de referência.


In [ ]:
import matplotlib.colors as mcolors
# Definição das colunas
categorias = ['Numerico','Categorico']
Nomecategorico = []
NomeNumerico = []
for column in df.columns:
    if df[column].dtype == 'object':
        Nomecategorico.append(column)
    else:
        NomeNumerico.append(column)
Nomecategorico.append('Data')

categorico = df[Nomecategorico]
numerico = df[NomeNumerico]




# Dados para as categorias principais
categorias = ['Numérico', 'Categórico']
valores_categorias = [len(numerico.columns), len(categorico.columns)]

# Dados para as subcategorias
subcategorias = numerico.columns.tolist() + categorico.columns.tolist()
valores_subcategorias = [1] * len(subcategorias)

# Cores para as categorias principais e subcategorias
cores_categorias = ['#FF0000', '#0000FF']  # Vermelho para Numérico, Azul para Categórico
cores_subcategorias = [
    mcolors.to_rgba('#FF0000', alpha=0.6)] * len(numerico.columns) + \
    [mcolors.to_rgba('#0000FF', alpha=0.6)] * len(categorico.columns)

# Tamanho da figura (aumentado para 22 x 14)
fig, ax = plt.subplots(figsize=(22, 14))

# Gráfico das categorias principais
p1 = ax.pie(valores_categorias, 
            radius=1.2,
            labels=categorias,
            wedgeprops=dict(edgecolor='white'),
            colors=cores_categorias,
            textprops=dict(color="black", fontweight="bold"))

# Gráfico das subcategorias (nomes das colunas)
p2 = ax.pie(valores_subcategorias,
            radius=1,  # Aumentando o raio interno para criar mais espaço
            labels=subcategorias,
            autopct=None,
            colors=cores_subcategorias,
            labeldistance=0.6,  # Aumentando a distância do texto para melhorar o encaixe
            wedgeprops=dict(edgecolor='white'), 
            pctdistance=0.75,  
            rotatelabels=True,
            textprops=dict(color="white", fontweight="bold"))

# Limpa o centro do círculo
centre_circle = plt.Circle((0, 0), 0.5, fc='white')  # Reduzindo o círculo central para mais espaço

# Adiciona o círculo central
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Título
plt.title('Distribuição das Colunas entre Numérico e Categórico')

# Adiciona a legenda
plt.legend(labels=['Numérico', 'Categórico'], loc='upper right', fontsize=14)

plt.show()




### Conclusão
O gráfico de pizza evidencia a predominância de colunas numéricas no conjunto de dados em relação às colunas categóricas. Essa predominância indica que o DataFrame é composto majoritariamente por variáveis quantitativas, o que é útil em análises que envolvem cálculos matemáticos, modelagem estatística e aprendizado de máquina.

### Matriz de Corelação

In [ ]:
df_analise = df
df_analise = df_analise.drop(['Desconto R$', 'Desc %'], axis=1)

df_analise_radio = df_analise[df_analise['UEN'] == 'Rádio']
df_analise_televisao = df_analise[df_analise['UEN'] == 'Televisão']
df_analise_digital = df_analise[df_analise['UEN'] == 'Digital']


In [ ]:

# Agrupamento mensal por 'Cliente' mantendo o 'Setor'
df_analise_radio = df_analise_radio.groupby([pd.Grouper(key='Data', freq='M'), 'Cliente', 'Setor']).agg({
    'VL Tabela': 'sum',
    'Vl Bruto': 'sum',
    'Vl Liquido Final': 'sum',
    'IPCA BR': 'mean',
    'IPCA ES': 'mean',
    'Taxa Ac. TRI % PIB': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR': 'mean'
}).reset_index()


In [ ]:
df_analise_radio.head()

In [ ]:
df_analise_radio.info()

In [ ]:
df_analise_radio_numericos = df_analise_radio.drop(['Data', 'Cliente', 'Setor'], axis=1)


In [ ]:
correlation_matrix = df_analise_radio_numericos.corr()

# Configura o tamanho da figura
plt.figure(figsize=(12, 8))

# Cria o heatmap usando seaborn
sns.heatmap(correlation_matrix, 
            annot=True,           # Mostra os valores na matriz
            fmt=".2f",            # Formato dos números
            cmap='coolwarm',      # Escolhe a paleta de cores
            linewidths=0.5,       # Largura das linhas que separam os quadrados
            square=True,          # Quadrados perfeitos
            cbar_kws={"shrink": .75})  # Ajuste do tamanho da barra de cores

# Título do gráfico
plt.title('Matriz de Correlação - Análise Rádio', fontsize=16)

# Exibe o gráfico
plt.show()

In [ ]:
df_analise_radio_mes = df_analise_radio.drop(['Cliente', 'Setor'], axis=1)

# Agrupamento mensal pela coluna 'Data' e aplicação das funções de agregação
df_grouped = df_analise_radio_mes.groupby(pd.Grouper(key='Data', freq='M')).agg({
    'VL Tabela': 'sum',  # Soma dos valores financeiros
    'Vl Bruto': 'sum',   
    'Vl Liquido Final': 'sum',
    'IPCA BR': 'mean',   # Média dos indicadores econômicos
    'IPCA ES': 'mean',
    'Taxa Ac. TRI % PIB': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR': 'mean'
}).reset_index()


In [ ]:
df_grouped.head()

In [ ]:
df_grouped.info()

In [ ]:
df_grouped_numeric = df_grouped.drop(['Data'], axis=1)
correlation_matrix = df_grouped_numeric.corr()

# Configura o tamanho da figura
plt.figure(figsize=(12, 8))

# Cria o heatmap usando seaborn
sns.heatmap(correlation_matrix, 
            annot=True,           # Mostra os valores na matriz
            fmt=".2f",            # Formato dos números
            cmap='coolwarm',      # Escolhe a paleta de cores
            linewidths=0.5,       # Largura das linhas que separam os quadrados
            square=True,          # Quadrados perfeitos
            cbar_kws={"shrink": .75})  # Ajuste do tamanho da barra de cores

# Título do gráfico
plt.title('Matriz de Correlação - Análise Rádio', fontsize=16)

# Exibe o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot do primeiro eixo y (Vl Liquido Final)
ax1.plot(df_grouped['Data'], df_grouped['Vl Liquido Final'], color='blue', marker='o', label='Vl Liquido Final')
ax1.set_xlabel('Data')  # Rótulo do eixo x
ax1.set_ylabel('Vl Liquido Final', color='blue')  # Rótulo do eixo y esquerdo
ax1.tick_params(axis='y', labelcolor='blue')  # Cor dos rótulos do eixo y esquerdo

# Criação do segundo eixo y
ax2 = ax1.twinx()
ax2.plot(df_grouped['Data'], df_grouped['IPCA ES'], color='red', marker='x', label='IPCA ES')
ax2.set_ylabel('IPCA ES', color='red')  # Rótulo do eixo y direito
ax2.tick_params(axis='y', labelcolor='red')  # Cor dos rótulos do eixo y direito

# Título do gráfico
plt.title('Vl Liquido Final vs IPCA ES ao longo do Tempo')

# Legendas
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Exibe o gráfico
plt.show()

### Plotagem de gráficos de Clientes por frequência

In [ ]:
client_counts = df['Cliente'].value_counts()
top_clients = client_counts.head(20)

# Configuração do gráfico
plt.figure(figsize=(22, 12))

# Gerar cores diferentes para cada cliente
colors = plt.cm.get_cmap('tab20', len(top_clients))

# Criar o gráfico de barras
bars = plt.bar(range(len(top_clients)), top_clients, color=colors(range(len(top_clients))))

# Título e rótulos
plt.title('Top 20 Clientes por Frequência', fontsize=16)
plt.xlabel('Clientes', fontsize=14)
plt.ylabel('Contagem de Frequência', fontsize=14)
plt.xticks(ticks=range(len(top_clients)), labels=['']*len(top_clients), fontsize=12)

# Adicionar uma legenda abaixo do gráfico
plt.legend(bars, top_clients.index, title="Clientes", bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=4, fontsize=12)

# Ajustar o layout
plt.tight_layout()
plt.show()

#### Conclusão: Recorrência de Clientes nas Campanhas Publicitárias
A análise dos gráficos revela a presença significativa de clientes recorrentes que investem em diversas campanhas publicitárias simultaneamente em diferentes veículos e Unidades de Negócio ao longo de um mesmo mês. Essa recorrência destaca a confiança desses clientes na eficácia das estratégias de publicidade oferecidas, bem como a importância de manter um relacionamento contínuo e estratégico com esses parceiros.

In [ ]:
df[df['Cliente'] == 'SUPERINTENDENCIA ESTADUAL DE COMUNICACAO']

### Plotagem de gráficos de Frequência de um Setor por UEN
aqui podemos observar que existe uma tendência a um determinado setor ser mais predominante naquele UEN

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gerando o DataFrame com as contagens para cada UEN e Setor
df_counts = df.groupby(['UEN', 'Setor']).size().reset_index(name='counts')

# Garantindo que cada UEN tenha 10 setores representados, mesmo em caso de empate
df_counts = df_counts.groupby('UEN').apply(lambda x: x.nlargest(10, 'counts', keep='all')).reset_index(drop=True)

# Paleta de cores personalizada para garantir que as cores sejam consistentes entre os gráficos
unique_setores = df_counts['Setor'].unique()
palette = dict(zip(unique_setores, sns.color_palette("tab20", len(unique_setores))))

# Iterando sobre cada UEN e criando um gráfico separado
for uen, group_data in df_counts.groupby('UEN'):
    plt.figure(figsize=(9, 8))
    
    # Criando o gráfico de barras
    ax = sns.barplot(x='Setor', y='counts', data=group_data, palette=palette)
    
    # Adicionando os rótulos em cima das barras
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()-50),
                    ha = 'center', va = 'center',
                    xytext = (0, 9), textcoords = 'offset points')
    
    # Adicionando o nome do Setor abaixo da barra
    ax.set_xticks(range(len(group_data['Setor'])))
    ax.set_xticklabels(group_data['Setor'], rotation=45, ha='right')

    # Configurações adicionais de título e formatação
    ax.set_xlabel('Setores')
    ax.set_ylabel('Contagem de Frequência')
    ax.set_title(f'UEN: {uen}', size=16, weight='semibold')

    # Ajustando o layout
    plt.tight_layout()

    # Exibir o gráfico
    plt.show()



#### Conclusão: Predominância do Setor de Loja Varejista/Atacado
A análise dos gráficos revela um domínio claro do setor de Loja Varejista/Atacado na maioria das Unidades de Negócio da Rede Gazeta. Este setor lidera em quase todas as áreas, exceto no Digital, onde os Prestadores de Serviços assumem a liderança. Essa predominância sugere que a maior parte das campanhas publicitárias da Rede Gazeta é direcionada a clientes do setor varejista, refletindo a importância desse segmento na estratégia comercial da empresa.

A concentração de campanhas publicitárias voltadas para o setor de varejo indica uma demanda significativa e uma dependência estratégica desse segmento para a geração de receita. Esse comportamento pode ser explicado pela grande presença e relevância do varejo no mercado local, além de sua capacidade de investir em publicidade para alcançar um público amplo e diversificado.

Por outro lado, o destaque dos Prestadores de Serviços no setor Digital sugere uma tendência diferente nesse canal, possivelmente ligada à natureza dos serviços oferecidos e à necessidade de uma presença online mais forte e direcionada. Esse dado ressalta a importância de uma abordagem diferenciada para o segmento de serviços, especialmente em plataformas digitais.

Compreender essa distribuição setorial permite à Rede Gazeta ajustar suas estratégias de marketing e vendas de acordo com as necessidades e comportamentos específicos de cada setor. Além disso, identificar áreas de menor presença pode abrir oportunidades para expandir a atuação em outros segmentos, diversificando o portfólio de clientes e reduzindo a dependência de um único setor.

In [ ]:
dados = {
    'Mês': [],
    'Receita Líquida': []
}
for mes in df['Data'].astype(str).str[:7].unique():
    receita_mes = df['Vl Liquido Final'].where(df['Data'].astype(str).str[:7] == mes)
    receita_mes_total = receita_mes.sum()
    dados['Mês'].append(mes)
    dados['Receita Líquida'].append(receita_mes_total)
dados = pd.DataFrame(dados)
dados['Mês'] = pd.to_datetime(dados['Mês'], format='%Y-%m')
dados = dados.sort_values('Mês').reset_index(drop=True)
# Adicionando a coluna 'Ano' para facilitar a diferenciação
dados['Ano'] = dados['Mês'].dt.year
# Plotando o gráfico com cores diferentes para cada ano
plt.figure(figsize=(12, 6))
sns.lineplot(data=dados, x='Mês', y='Receita Líquida', hue='Ano', palette='tab10', marker='o')
# Configurações do gráfico
plt.title('Variação da Receita Mês a Mês')
plt.xlabel('Mês')
plt.ylabel('Receita Líquida em R$5.000.000')
plt.grid(True)
plt.yticks(np.arange(0, dados['Receita Líquida'].max(), 5000000))
plt.xticks(rotation=45)
plt.tight_layout()
# Adicionando as linhas verticais de mudança de ano
for i, row in dados.iterrows():
    if i > 0 and row['Ano'] != dados.loc[i-1, 'Ano']:
        plt.axvline(x=row['Mês'], color='red', linestyle='--')
plt.legend(title='Ano')
plt.show()

In [ ]:
veiculos = df['UEN'].unique()
for veiculo in veiculos:
    dados_veiculo = {
        'Mês': [],
        'Receita Líquida': []
    }
    for mes in df['Data'].astype(str).str[:7].unique():
        receita_mes = df.loc[df['UEN'] == veiculo, 'Vl Liquido Final'].where(df['Data'].astype(str).str[:7] == mes)
        receita_mes_total = receita_mes.sum()
        dados_veiculo['Mês'].append(mes)
        dados_veiculo['Receita Líquida'].append(receita_mes_total)
    dados_veiculo = pd.DataFrame(dados_veiculo)
    dados_veiculo['Mês'] = pd.to_datetime(dados_veiculo['Mês'], format='%Y-%m')
    dados_veiculo = dados_veiculo.sort_values('Mês').reset_index(drop=True)
    # Adicionando a coluna 'Ano' para facilitar a diferenciação
    dados_veiculo['Ano'] = dados_veiculo['Mês'].dt.year
    # Plotando o gráfico para o veículo específico
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=dados_veiculo, x='Mês', y='Receita Líquida', hue='Ano', palette='tab10', marker='o')
    # Configurações do gráfico
    plt.title(f'Variação da Receita Mês a Mês para o Veículo: {veiculo}')
    plt.xlabel('Mês')
    plt.ylabel('Receita Líquida por R$1.000.000')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.yticks(np.arange(0, dados_veiculo['Receita Líquida'].max(), 1000000))
    plt.tight_layout()
    # Adicionando as linhas verticais de mudança de ano
    for i, row in dados_veiculo.iterrows():
        if i > 0 and row['Ano'] != dados_veiculo.loc[i-1, 'Ano']:
            plt.axvline(x=row['Mês'], color='red', linestyle='--')
    plt.legend(title='Ano')
    plt.show()

### Análise da Variação de Receita por Sazonalidade

A análise dos gráficos revela um padrão claro na variação de receita ao longo do ano nas Unidades de Negócio. No primeiro semestre, observa-se um aumento gradual na receita, que pode ser atribuído ao impulso inicial das atividades econômicas no início do ano. No entanto, ao iniciar o segundo semestre, há uma tendência de queda na receita, seguida por uma recuperação nos meses subsequentes. Esse comportamento resulta em um pico significativo no mês de dezembro.

Essa variação indica que a sazonalidade tem um papel crucial na geração de receita, com eventos, feriados e outras ocasiões específicas impactando diretamente a frequência e o volume das transações comerciais. A recorrência desses picos anuais sugere que as empresas devem estar preparadas para aproveitar esses períodos de alta demanda, ao mesmo tempo em que desenvolvem estratégias para mitigar as quedas observadas no meio do ano.

Por isso, forma-se a hipótese de que a sazonalidade é um fator determinante na variação de receita, e entender esse padrão permite uma melhor alocação de recursos e planejamento estratégico para maximizar o desempenho financeiro ao longo do ano.



### Plotagem de gráfico UEN por frequência    

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.keys()

In [ ]:

plt.figure(figsize=(10,6))
plt.hist(df['UEN'], bins=20, color='blue', edgecolor='black')
plt.title('Frequência da tabela')
plt.xlabel('UEN')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()


### Plotagem de gráfico Top 10 Clientes por ano

### Introdução
O gráfico de linhas acima apresenta a evolução da frequência dos principais clientes ao longo dos anos, destacando os 10 clientes mais recorrentes em um período específico. Cada linha no gráfico representa um cliente e mostra como a frequência de suas transações, ou outras métricas relevantes, variou ano a ano. O objetivo é identificar quais clientes mantiveram uma posição de destaque ao longo do tempo, sugerindo estabilidade, crescimento ou declínio em suas interações.

In [ ]:

# Extraindo o ano da coluna Data
df['Ano'] = df['Data'].dt.year

# Contando a frequência de cada cliente por ano
frequencia_por_cliente = df.groupby(['Ano', 'Cliente']).size().unstack(fill_value=0)

# Selecionando os top 10 clientes por ano
top_clientes_por_ano = frequencia_por_cliente.apply(lambda x: x.nlargest(10), axis=1)

# Plotando o gráfico
plt.figure(figsize=(14,8))

for cliente in top_clientes_por_ano.columns:
    plt.plot(top_clientes_por_ano.index, top_clientes_por_ano[cliente], label=cliente)

plt.title('Top Clientes por Ano')
plt.xlabel('Ano')
plt.ylabel('Frequência')
plt.legend(title='Cliente', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()


### Conclusão
A partir do gráfico, é possível observar que alguns clientes, como "ASSOCIACAO DE SUPERMERCADOS REDE SHOW" e "COMPANHIA DO BOI COMERCIO DE CARNES EIRE," mantêm uma presença constante no top 10 ao longo de vários anos, demonstrando uma relação duradoura e consistente. Por outro lado, outros clientes aparecem apenas esporadicamente no top 10, indicando um comportamento de interação menos regular ou sazonal. Esta análise é crucial para entender a fidelidade dos clientes, identificar oportunidades de retenção e crescimento e adaptar estratégias comerciais para diferentes perfis de clientes.

#### Gráfico de Recorrência por desconto médio

**Introdução:**

Neste estudo, buscamos entender a relação entre o percentual de desconto oferecido pela nossa empresa e a recorrência dos clientes em fechar contratos. A hipótese inicial era de que maiores descontos poderiam resultar em uma maior fidelização dos clientes, incentivando-os a fechar mais contratos ao longo do tempo.

In [ ]:
average_discounts = df.groupby('Cliente')['Desc %'].mean()

recurrence = df['Cliente'].value_counts()

result_df = pd.DataFrame({
    'Recorrencia': recurrence,
    'Desconto Médio (%)': average_discounts
}).reset_index()

result_df.columns = ['Cliente', 'Recorrencia', 'Desconto Médio (%)']

plt.figure(figsize=(10, 6))
plt.scatter(result_df['Desconto Médio (%)'], result_df['Recorrencia'])

for i, row in result_df.iterrows():
    plt.text(row['Desconto Médio (%)'], row['Recorrencia'], '.', fontsize=9)

plt.title('Recorrência de Clientes x Desconto Médio (%)')
plt.xlabel('Desconto Médio (%)')
plt.ylabel('Recorrência')
plt.grid(True)
plt.show()
plt.show()


**Conclusão:**

Após analisar os dados e visualizar a recorrência dos clientes em relação à média de desconto recebido, constatamos que não há uma correlação clara entre esses dois fatores. Alguns clientes que fecharam um grande número de contratos não receberam qualquer desconto, enquanto outros que receberam descontos não demonstraram maior recorrência. Isso sugere que outros fatores, além do desconto, podem estar influenciando a fidelização dos clientes, e que o desconto, por si só, não é um determinante da lealdade do cliente na Rede Gazeta.

#### Gráficos de IPCA por quantidade de contratos fechados

**Introdução:**

O objetivo desta análise foi investigar se a variação do Índice de Preços ao Consumidor Amplo (IPCA) no Brasil influencia a quantidade de contratos fechados pela Rede Gazeta. A hipótese inicial era de que um IPCA mais baixo estaria associado a um maior número de contratos, indicando que períodos de menor inflação poderiam estimular a atividade comercial e, consequentemente, a contratação de serviços.

In [ ]:
bins = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1', '1-1.25', '1.25-1.5', '1.5-1.75', '1.75-2']

# Criar uma nova coluna categorizando os valores de IPCA ES nos intervalos definidos
df['IPCA ES Interval'] = pd.cut(df['IPCA ES'], bins=bins, labels=labels, right=False)

# Contar a quantidade de contratos para cada intervalo de IPCA ES
contratos_por_intervalo = df['IPCA ES Interval'].value_counts().sort_index()

# Plotar o gráfico
plt.figure(figsize=(10, 6))
plt.bar(contratos_por_intervalo.index, contratos_por_intervalo, width=0.5)

# Ajustes do gráfico
plt.xlim(-0.5, len(labels)-0.5)
plt.ylim(0, 14000)  # Limites do eixo Y de 0 a 14000 conforme sua solicitação
plt.title('Quantidade de Contratos Fechados por Intervalo de IPCA ES')
plt.xlabel('Intervalo de IPCA ES')
plt.ylabel('Quantidade de Contratos')
plt.grid(True, axis='y')

# Exibir o gráfico
plt.show()



In [ ]:
# Definir os intervalos desejados
bins = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1', '1-1.25', '1.25-1.5', '1.5-1.75', '1.75-2']

# Criar uma nova coluna categorizando os valores de IPCA ES nos intervalos definidos
df['IPCA BR Interval'] = pd.cut(df['IPCA BR'], bins=bins, labels=labels, right=False)

# Contar a quantidade de contratos para cada intervalo de IPCA ES
contratos_por_intervalo = df['IPCA BR Interval'].value_counts().sort_index()

# Plotar o gráfico
plt.figure(figsize=(10, 6))
plt.bar(contratos_por_intervalo.index, contratos_por_intervalo, width=0.5)

# Ajustes do gráfico
plt.xlim(-0.5, len(labels)-0.5)
plt.ylim(0, 14000)  # Limites do eixo Y de 0 a 14000 conforme sua solicitação
plt.title('Quantidade de Contratos Fechados por Intervalo de IPCA BR')
plt.xlabel('Intervalo de IPCA BR')
plt.ylabel('Quantidade de Contratos')
plt.grid(True, axis='y')

# Exibir o gráfico
plt.show()

**Conclusão:**

Ao analisar o gráfico da quantidade de contratos fechados em função do IPCA BR, inicialmente observei uma aparente correlação entre um IPCA mais baixo e um maior número de contratos fechados. No entanto, ao não considerar os meses e anos correspondentes, a análise perdeu parte de sua relevância e precisão. Isso ressaltou a importância de incorporar a dimensão temporal em análises econômicas, para que se possa entender melhor a influência de variáveis como a inflação ao longo do tempo, ao invés de se basear em observações pontuais.

## Plotagem gráfico Top 20 Clientes por Valor de Vl Liquido Final

Este gráfico ilustra a contribuição de cada cliente para a receita total gerada. Ele é especialmente útil quando comparado ao gráfico de relação entre cliente e frequência, permitindo identificar clientes que, independentemente da alta ou baixa frequência de interações, fecham contratos de maior valor. Isso pode ajudar a destacar clientes estratégicos que impulsionam significativamente a receita da empresa.

In [ ]:
import matplotlib.pyplot as plt

# Supondo que 'Cliente' seja a coluna com os nomes dos clientes e 'Vl Liquido Final' seja a coluna com os valores
client_sums = df.groupby('Cliente')['Vl Liquido Final'].sum()
top_clients = client_sums.nlargest(20)

# Configuração do gráfico
plt.figure(figsize=(22, 12))

# Gerar cores diferentes para cada cliente
colors = plt.cm.get_cmap('tab20', len(top_clients))

# Criar o gráfico de barras
bars = plt.bar(range(len(top_clients)), top_clients, color=colors(range(len(top_clients))))

# Título e rótulos
plt.title('Top 20 Clientes por Valor de Vl Liquido Final', fontsize=16)
plt.xlabel('Clientes', fontsize=14)
plt.ylabel('Soma de Vl Liquido Final', fontsize=14)
plt.xticks(ticks=range(len(top_clients)), labels=['']*len(top_clients), fontsize=12)

# Adicionar uma legenda abaixo do gráfico
plt.legend(bars, top_clients.index, title="Clientes", bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=4, fontsize=12)

# Ajustar o layout
plt.tight_layout()
plt.show()


#### Conclusão

A partir dos gráficos analisados, podemos observar que alguns clientes, como a "REALMAR DISTRIBUIDORA LTDA" e a "SUPERINTENDENCIA ESTADUAL DE COMUNICACAO", são os principais responsáveis pela maior parte da receita gerada, apesar de outros clientes terem uma frequência de interação igualmente alta. Isso sugere que a geração de receita não está necessariamente relacionada ao número de transações, mas possivelmente ao valor de cada contrato fechado. 

Uma hipótese a ser explorada é que clientes com menor frequência de interação, mas que aparecem no topo da lista de receita, podem estar realizando contratos de maior valor unitário. Esse insight pode direcionar a estratégia comercial para focar em aumentar o valor das transações com clientes de alta frequência ou em fortalecer o relacionamento com clientes que já trazem grande valor, mesmo com interações menos frequentes.